In [8]:
import os
import requests
import datetime
from zipfile import ZipFile
import ftplib 
import cdsapi
import yaml
import xarray as xr
import glob

In [9]:
#This file contains configuration details like API keys and passwords
global_vars = yaml.safe_load(open('../config.yml', 'r') )

In [14]:
#This has custom functions - specifically the "download" function 
%run ./00_custom_functions.ipynb

In [11]:
#Data will be saved at this location in folders such as root/SST/originals/ and root/SSS/orginals/
#Note that these folder locations must already be created as a manual validation step
download_folder_root = global_vars['download_folder']
print(download_folder_root)

/data/artemis/workspace/ds4114/online_data/


In [12]:
#The following two variables are used to acquire select data when it is uploaded by month or year.
    #Data that is not uploaded by year includes: SST (NOAA); MLD (deBoyer & Argo), fCO2 (SOCAT), xCO2, Coastal, SeaFlux
#These set the start and end years (inclusive) and do not need to be changed.
#Some years/months of data may not available (because prior to when data was gathered or too recent for the source).
    #In those cases any available data is obtained in this range. Specifically, 
        #SST (NOAA) data only 1981-present
        #SST (ERA5) data only 1979-present
        #SST (JRA55) data only 1958-2023
        #SSS data only 1900-present
        #MLD (deBoyer and Argo) data only an averaged 12 months
        #CHL data only 1997-present
        #fCO2 data only 1970-2022
        #SLP data only 1979-2022
        #xCO2 data only 1979-present
        #Coastal data only an averaged 12 months 
        #SeaFlux data only 1982-2022
    #These limitations are hardcoded so other sources or links would be needed to download outside of this range 
acquisition_start_year = 1979 
acquisition_end_year = 2023  

#This variable sets the output filetype for SSS and CHL data and needs to specified explicitly because of the unique way the are downloaded.
#By default, all data source downloads will default to the netcdf format unless using cloud storage. 
#When using cloud storage, it is recommended to use ARCO (Analysis-Ready Cloud-Optimized) formats like Zarr over NetCDF
output_file_type = '.zarr' if download_folder_root[0:5] == 'gs://' else '.nc'

#This variable is used for naming files (SST & xCO2)
today_yearmonth = datetime.datetime.now().strftime('%Y%m')

In [6]:
#Some data (SLP, SST) requires an account on European Centre for Medium-Range Weather Forecasts and an API key
#More info can be found https://cds.climate.copernicus.eu/api-how-to
#Once the packages is installed (conda install -c conda-forge cdsapi), we need to install your API key using code below

cds_url= "url: https://cds.climate.copernicus.eu/api/v2"
cds_key= 'key: '+global_vars['cds_api_key']  #from the configuration file
file = os.path.expanduser('~')+'/.cdsapirc'
if not (os.path.isfile(file)): 
    cds_file = open(file, "w")
    cds_file.write(cds_url+'\n')
    cds_file.write(cds_key)
    cds_file.close()
else:
    print("API key already installed")

API key already installed


## Temperature (SST)
#### NOAA

In [7]:
#NOAA SST is easy because NOAA uses the same URL for all data across time periods (1981-09 to present only)
#More info found on https://psl.noaa.gov/data/gridded/data.noaa.oisst.v2.highres.html
sst_direct_url = 'https://downloads.psl.noaa.gov/Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc' 
sst_destination_folder = download_folder_root+r'SST/originals/'
sst_destination_filename = 'SST_NOAA_OI-V2-HighRes_198109-'+today_yearmonth+'.nc'  #data is only from 1981
download(sst_direct_url, sst_destination_folder, sst_destination_filename)

File SST_NOAA_OI-V2-HighRes_198109-202308.zarr already exists at gs://leap-persistent/ds4114/online_data/SST/originals/ - (skipping download from https://downloads.psl.noaa.gov/Datasets/noaa.oisst.v2.highres/sst.mon.mean.nc )


#### ECMWF

In [15]:
#This SST dataset comes from European Centre for Medium-Range Weather Forecasts (1979-01 to present only). It is the same as the one used for SLP below.
#More info can be found on https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#We installed the API and have an account, so now we can download the ERA5 data via a loop (yearly)
#While this data can be downloaded together with SLP, we have chosen to break up the downloads for clarity and organization
sst_destination_folder = download_folder_root+r'SST/originals/'
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    sst_destination_filename = ''
    months = []
    if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
        months = [i for i in range(1,datetime.datetime.now().month -1)]  
        sst_destination_filename = 'SST_ECMWF_ERA5-monthly-reanalysis-SST_'+str(year)+'.nc'  
        #Note, partial years will not automatically be overwritten so you must manually clean up unneeded files
    else:
        sst_destination_filename = 'SST_ECMWF_ERA5-monthly-reanalysis-SST_'+str(year)+'.nc'
        months = [i for i in range(1,12+1)]

    cdsapi_custom_download(year, months, 'sea_surface_temperature', sst_destination_folder, sst_destination_filename, overwrite=False, create_dest=False)
    
print("ERA5 SST Complete")

File SST_ECMWF_ERA5-monthly-reanalysis-SST_1979.nc already exists - (skipping download from 1979 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1980.nc already exists - (skipping download from 1980 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1981.nc already exists - (skipping download from 1981 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1982.nc already exists - (skipping download from 1982 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1983.nc already exists - (skipping download from 1983 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1984.nc already exists - (skipping download from 1984 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1985.nc already exists - (skipping download from 1985 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1986.nc already exists - (skipping download from 1986 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1987.nc already exists - (skipping download from 1987 )
File SST_ECMWF_ERA5-monthly-reanalysis-SST_1988.nc already exists - (skipping download from 1988 )
File SST_E

2023-08-28 14:05:43,444 INFO Welcome to the CDS
2023-08-28 14:05:43,446 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-08-28 14:05:43,772 INFO Request is completed
2023-08-28 14:05:43,774 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data7/adaptor.mars.internal-1693231843.3536522-7842-17-244bef39-e605-4dd6-ab69-958c74f39fae.nc to ../tmp/SST_ECMWF_ERA5-monthly-reanalysis-SST_2023.nc (11.9M)
2023-08-28 14:05:53,632 INFO Download rate 1.2M/s   


Transferring to Destination /data/artemis/workspace/ds4114/online_data/SST/originals/SST_ECMWF_ERA5-monthly-reanalysis-SST_2023.nc...
ERA5 SST Complete


#### JRA55

In [12]:
#This SST data source comes from Japan Meteorological Agency (JMA) (195801 - to 2023 only)
#JRA55-do (Tsujino et al., 2018)  corrects the atmospheric reanalysis product JRA-55 (Kobayashi et al., 2015) 
#More info can be found on https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/

sst_destination_folder = download_folder_root+r'SST/originals/'
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    sst_jra55_destination_filename = 'SST_JMA_JRA55-do-daily-reanalysis-SST_'+str(year)+'.nc'
    direct_link = None
    
    #downloading via direct links. Recent data (2020+) is hosted on a different site
    if year == 2020:
        direct_link = 'https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/ocean/day/tos/gn/v20210315/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0-1_gn_20200101-20201231.nc'
    elif year == 2021:
        direct_link = 'https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/ocean/day/tos/gn/latest/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0-1_gn_20210101-20211231.nc'
    elif year == 2022:
        direct_link = 'https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/ocean/day/tos/gn/latest/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0-1_gn_20220101-20221231.nc'
    elif year == 2023:
        direct_link = 'https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/ocean/day/tos/gn/latest/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0-1_gn_20230101-20230824.nc'
            #previously: #direct_link = 'https://climate.mri-jma.go.jp/pub/ocean/JRA55-do/ocean/day/tos/gn/latest/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0-1_gn_20230101-20230629.nc'
        sst_jra55_destination_filename = 'SST_JMA_JRA55-do-daily-reanalysis-SST_'+str(year)+'.nc' #rename if desired since only part of the year available
    else:
        base = 'https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-5-0/ocean/day/tos/gn/v20200916/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gn_'
        end = str(year)+'0101-'+str(year)+'1231.nc'
        direct_link = base + end  #ex: #https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-5-0/ocean/day/tos/gn/v20200916/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gn_19580101-19581231.nc
    
    download(direct_link, sst_destination_folder, sst_jra55_destination_filename)
print("JRA55 SST Complete")

File SST_JMA_JRA55-do-daily-reanalysis-SST_1979.zarr already exists at gs://leap-persistent/ds4114/online_data/SST/originals/ - (skipping download from https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-5-0/ocean/day/tos/gn/v20200916/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gn_19790101-19791231.nc )
File SST_JMA_JRA55-do-daily-reanalysis-SST_1980.zarr already exists at gs://leap-persistent/ds4114/online_data/SST/originals/ - (skipping download from https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-5-0/ocean/day/tos/gn/v20200916/tos_input4MIPs_atmosphericState_OMIP_MRI-JRA55-do-1-5-0_gn_19800101-19801231.nc )
File SST_JMA_JRA55-do-daily-reanalysis-SST_1981.zarr already exists at gs://leap-persistent/ds4114/online_data/SST/originals/ - (skipping download from https://esgf-data2.llnl.gov/thredds/fileServer/user_pub_work/input4MIPs/CMIP6/OMIP/MRI/MRI-JRA55-do-1-5-0/ocean/da

## Salinity (SSS)

In [ ]:
#SST requires downloading yearly zip files and extracting each month from each  (1900-01 to present only)
#More info found on https://www.metoffice.gov.uk/hadobs/en4/download-en4-2-2.html
#Note: because the file names in the zip are not known until extracting, this cell will execute even if the data already has been downloaded

sss_direct_url_base = 'https://www.metoffice.gov.uk/hadobs/en4/data/en4-2-1/EN.4.2.2/'
sss_destination_filename_base = 'SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_' #_197901+
sss_destination_folder = download_folder_root+r'SSS/originals/'
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    if year >= 2021: sss_direct_url_base = 'https://www.metoffice.gov.uk/hadobs/en4/data/en4-2-1/' #2021 and onward the URL changes, though both URL zips exists. Not clear why.
    url_file = 'EN.4.2.2.analyses.g10.'+str(year)+'.zip'
    sss_direct_url = sss_direct_url_base + url_file
    sss_destination_filename = sss_destination_filename_base+str(year)+'.zip'
    
    #download to tmp, extract a year, then transfer all to destination
    tmp_folder = '../tmp/'
    download_to_folder(sss_direct_url, tmp_folder, sss_destination_filename,overwrite=False, create_dest=True)
    ZipFile(tmp_folder+sss_destination_filename).extractall(tmp_folder)
    files = glob.glob(tmp_folder+'EN.4.2.2.f.analysis.g10.*.nc')
    for f in files:  #for each file extracted
        basename = os.path.basename(f)
        tmp_xr = xr.open_dataset(tmp_folder + basename)
        output_xarray_with_date(tmp_xr, sss_destination_folder, basename, filetype=output_file_type, with_date=False) 
            #Note the filename is kept as the original from the .zip here to be compatible with other work. To output the file with a different name, edit the dest_file string parameter.
        os.remove(os.path.join(tmp_folder,f)) #remove tmp file
    
    #remove zip file too
    os.remove(os.path.join(tmp_folder,sss_destination_filename)) #remove tmp file
print("SSS Complete")

## Mixed Layer Depth (MLD)
#### deBoyer

In [13]:
#MLD data is just one year of data that was processed by the author using several years of temperature profiles (data from 1941-2008, partially missing)
#More info found on https://cerweb.ifremer.fr/deboyer/mld/Surface_Mixed_Layer_Depth.php

mld_direct_url = 'https://cerweb.ifremer.fr/deboyer/data/mld_DT02_c1m_reg2.0.nc'
mld_destination_folder = download_folder_root+r'MLD/originals/'
mld_destination_filename = 'MLD_IFREMER-deBoyer_DT02-c1m_2008.nc'   #2008 though data is blended across years 
download(mld_direct_url, mld_destination_folder, mld_destination_filename)

File MLD_IFREMER-deBoyer_DT02-c1m_2008.zarr already exists at gs://leap-persistent/ds4114/online_data/MLD/originals/ - (skipping download from https://cerweb.ifremer.fr/deboyer/data/mld_DT02_c1m_reg2.0.nc )


#### Argo

In [14]:
# This MLD source is from UC San Diego who used Argo data to create the climatology
# More info can be found on http://mixedlayer.ucsd.edu/ and https://www.seanoe.org/data/00311/42182/#56126
mld_direct_url = 'http://mixedlayer.ucsd.edu/data/Argo_mixedlayers_monthlyclim_04142022.nc'
mld_destination_folder = download_folder_root+r'MLD/originals/'
mld_destination_filename = 'MLD_UCSD-Argo_mixedlayers-monthlyclim_2022.nc'   #2022 though data is blended across years 
download(mld_direct_url, mld_destination_folder, mld_destination_filename)

File MLD_UCSD-Argo_mixedlayers-monthlyclim_2022.zarr already exists at gs://leap-persistent/ds4114/online_data/MLD/originals/ - (skipping download from http://mixedlayer.ucsd.edu/data/Argo_mixedlayers_monthlyclim_04142022.nc )


## Chlorophyll (CHL)

In [ ]:
#CHL data needs to be downloaded via FTP in monthly files (1997-09 to present only)
#According to https://www.globcolour.info/products_description_mermet.html, we want to use GSM data only
#Note that the product changes over the years and the file names can vary. However there is a common pattern in naming used below.
#More info found on https://hermes.acri.fr/index.php?class=archive

#FTP requires an account and login:
chl_direct_ftp = 'ftp.hermes.acri.fr'
chl_destination_folder = download_folder_root+r'CHL/originals/'
chl_acquisition_start_year = acquisition_start_year if acquisition_start_year >= 1997 else 1997 #earliest we have is 1997-09 so set it here to prevent errors getting nonexistant data
usr = global_vars['chl_user'] 
psw = global_vars['chl_psw'] 

ftp_server = ftplib.FTP(chl_direct_ftp)
ftp_server.set_pasv(True)
ftp_server.login(usr, psw)

In [ ]:
#Note the nlst command used below may throw a [WinError 10060] message after logging in. This may be due to how the FTP server is configured or how the local computer is configured. 
#If the following command does not work, there is a broader issue with the connection.
ftp_server.dir()

In [ ]:
#Note, when using Google cloud storage, this cell will download data even if it already exists at the destination. This may be updated in a future version.

for year in range(chl_acquisition_start_year, acquisition_end_year+1, 1):
    chl_destination_filename = ''
    months = [i for i in range(1,12+1)]
     #if a partial year, reduce available months
    if year == datetime.datetime.now().year: months = [i for i in range(1,datetime.datetime.now().month)]
    elif year == 1997: months = [i for i in range(9,12+1)] #data only available part of 1997
    for month in months:
        chl_destination_filename = 'CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_'+str(year)+str(month).zfill(2)+'.nc'
        ftp_server.cwd(f'/GLOB/merged/month/{year}/{str(month).zfill(2)}/01') #ex. '/GLOB/merged/month/2022/02/01' for Feb 2022
        f = ftp_server.nlst('L3m*GLOB*100*GSM*CHL1*.nc')[0]    #Global view, 100 km resolution, GSM product, CHL1 data
        
        if (os.path.isfile(chl_destination_folder + chl_destination_filename):
            print(f"File {chl_destination_filename} already exists at {chl_destination_folder} - (skipping download from {url} )")
        else:
            #download to tmp folder first, then check and transfer to destination
            tmp_folder = '../tmp/'
            if not os.path.exists(tmp_folder): os.makedirs(tmp_folder)
            print(f'Downloading {f} as {chl_destination_filename}')
            ftp_server.retrbinary("RETR " + f, open(tmp_folder + chl_destination_filename, 'wb').write)
            tmp_xr = xr.open_dataset(tmp_folder + chl_destination_filename)
            output_xarray_with_date(tmp_xr, chl_destination_folder, chl_destination_filename, filetype=output_file_type, with_date=False)
            del tmp_xr
            os.remove(os.path.join(tmp_folder , chl_destination_filename)) #remove tmp file
            
print("CHL Complete")

In [22]:
ftp_server.quit()

'221 Goodbye.'

## pCO2 
### SOCAT fCO2

In [ ]:
#pCO2 data is calculated from fCO2 and SST (SOCOVV) and sea level pressure (ECMRWF) so we need to obtain a dataset for both

#fCO2 and SST comes from the Surface Ocean CO2 Variability and Vulnerability group (1970-01 to end of last year only)
#We want to use the same SST data points related to fCO2 from the same source (not using the separate NOAA SST here).
#More info can be found at https://www.socat.info/index.php/data-access/
fco2_direct_url = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0278913/SOCATv2023_Gridded_Dat/SOCATv2023_tracks_gridded_monthly.nc'  #thru 2022-12
      #previously 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0253659/SOCATv2022_Gridded_Dat/SOCATv2022_tracks_gridded_monthly.nc'  #thru 2021-12
fco2_destination_folder = download_folder_root+r'pCO2/originals/'    
fco2_destination_filename = 'fCO2_SOCOVV_SOCAT-gridded-monthly_2022.nc'    #filename year changed from source to denote data date, not publish date
download(fco2_direct_url, fco2_destination_folder, fco2_destination_filename)

### Coastal Filling (pCO2)

In [15]:
#This data set is used for enhancing coastal areas for flux calculation
#More info can be found https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:0209633 and https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0209633/
coastal_clim_direct = 'https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0209633/MPI-ULB-SOM_FFN_clim.nc'
coastal_clim_destination_folder = download_folder_root+r'pCO2/originals/' 
coastal_clim_filename = 'pCO2_NOAA-NCEI_MPI-ULB-SOM-FFN_1988-2020.nc'
download(coastal_clim_direct, coastal_clim_destination_folder, coastal_clim_filename)

File pCO2_NOAA-NCEI_MPI-ULB-SOM-FFN_1988-2020.zarr already exists at gs://leap-persistent/ds4114/online_data/pCO2/originals/ - (skipping download from https://www.ncei.noaa.gov/data/oceans/ncei/ocads/data/0209633/MPI-ULB-SOM_FFN_clim.nc )


## Sea Level Pressure (SLP)

In [10]:
#Sea level pressure comes from European Centre for Medium-Range Weather Forecasts (1979-01 to present only). It is the same source the one used for one SST above.
#More info can be found on https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#We installed the API and have an account, so now we can download the ERA5 data via a loop (yearly)
slp_destination_folder = download_folder_root+r'SLP/originals/'    
for year in range(acquisition_start_year, acquisition_end_year+1, 1):
    slp_destination_filename = ''
    months = []
    if year == datetime.datetime.now().year:   #if a partial year; this prevents erorrs trying to use the API to get future/non-existant data    
        months = [i for i in range(1,datetime.datetime.now().month)]  
        slp_destination_filename = 'SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_'+str(year)+'.nc'  #rename if desired for partial year
        #To consider, do we want to automatically overwrite parital years?
    else:
        slp_destination_filename = 'SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_'+str(year)+'.nc'
        months = [i for i in range(1,12+1)]

    cdsapi_custom_download(year, months, 'mean_sea_level_pressure', slp_destination_folder, slp_destination_filename, overwrite=False, create_dest=False)
    
print("SLP Complete")

File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1979.zarr already exists - (skipping download for 1979 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1980.zarr already exists - (skipping download for 1980 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1981.zarr already exists - (skipping download for 1981 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1982.zarr already exists - (skipping download for 1982 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1983.zarr already exists - (skipping download for 1983 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1984.zarr already exists - (skipping download for 1984 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1985.zarr already exists - (skipping download for 1985 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1986.zarr already exists - (skipping download for 1986 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1987.zarr already exists - (skipping download for 1987 )
File SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_1988.zarr already exists - (skipping download f

2023-08-28 14:17:03,589 INFO Welcome to the CDS
2023-08-28 14:17:03,590 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-08-28 14:17:03,872 INFO Request is queued
2023-08-28 14:17:05,001 INFO Request is running
2023-08-28 14:17:09,007 INFO Request is completed
2023-08-28 14:17:09,008 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data0/adaptor.mars.internal-1693232226.317367-24424-14-4c867e63-351c-4317-a25c-dd04cef54d3f.nc to ../tmp/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_2023.nc (27.7M)
2023-08-28 14:17:12,281 INFO Download rate 8.5M/s   


Transferring to GS gs://leap-persistent/ds4114/online_data/SLP/originals/SLP_ECMWF_ERA5-monthly-reanalysis-MSLP_2023.zarr...
SLP Complete


## Atmospheric CO2 (xCO2)

In [26]:
#xCO2 comes from NOAA. More information: https://gml.noaa.gov/ccgg/trends/gl_data.html

xco2_direct_url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.csv"
xco2_destination_folder = download_folder_root+r'xCO2/originals/'
today_yearmonth = datetime.datetime.now().strftime('%Y%m')
xco2_destination_filename = 'xCO2_NOAA_xCO2-mm-gl-monthly_197901-'+today_yearmonth+'.csv'
download(xco2_direct_url, xco2_destination_folder, xco2_destination_filename) 

Attempting to transfer non-ARCO file extension to cloud...
Saving https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_gl.csv to ../tmp/xCO2_NOAA_xCO2-mm-gl-monthly_197901-202308.csv...
Complete
Moved to GS gs://leap-persistent/ds4114/online_data/xCO2/originals/xCO2_NOAA_xCO2-mm-gl-monthly_197901-202308.csv.


## SeaFlux
#### All (Wind, Ice, Ocean Area)

In [28]:
#This data is is not required for pCO2 reconstruction but is used for calculating flux
#LDEO product is updated through the end of the previous year
#More information can be found https://zenodo.org/record/8099928

seaflux_all_direct = 'https://zenodo.org/record/8099928/files/SeaFlux.v2023.01_all_1982-2022.nc'
seaflux_destination_folder = download_folder_root+r'SeaFlux/originals/'
seaflux_destination_filename = 'SeaFlux_LDEO_SeaFlux-v202301-all_1982-2022.nc'
download(seaflux_all_direct, seaflux_destination_folder, seaflux_destination_filename)

File SeaFlux_LDEO_SeaFlux-v202301-all_1982-2022.nc already exists - (skipping download from https://zenodo.org/record/8099928/files/SeaFlux.v2023.01_all_1982-2022.nc )
